# Amazon scraping

In [ ]:
#! pip install requests
#! pip install beautifulsoup4
#! pip install matplotlib
#! pip install ipykernel
#! pip install requests-html
#! pip install selenium

In [ ]:
import utils_amazon
from importlib import reload
import os
import pandas as pd
import re
from datetime import datetime
import plotly.express as px

from IPython.display import display
pd.options.display.max_colwidth = 170
pd.options.display.max_rows = None

In [ ]:
parent_path = r"".replace("\\", "/")

In [ ]:
# About 50 mins
reload(utils_amazon)
df = utils_amazon.load_html_search()

In [ ]:
df.to_excel(os.path.join(parent_path,"file_name_{}_{}_{}.xlsx".format(datetime.today().year,datetime.today().month,datetime.today().day)), index=False)

In [ ]:
# Pass the DataFrame and copy and paste the product name you want to see in browser
reload(utils_amazon)
utils_amazon.open_link(df,'')

In [ ]:
# Pass the DataFrame and index you want to open in browser
reload(utils_amazon)
utils_amazon.open_link(df,4)

# Metrics

## Price Analysis

In [ ]:
df = pd.read_excel(os.path.join(parent_path,"file_name.xlsx".replace("\\", "/")))
print(df.shape)
df.head()

In [ ]:
reload(utils_amazon)
df_25 = utils_amazon.quartile_prices(df)[0]
df_25_50 = utils_amazon.quartile_prices(df)[1]
df_50_75 = utils_amazon.quartile_prices(df)[2]
df_75 = utils_amazon.quartile_prices(df)[3]
quartiles = utils_amazon.quartile_prices(df)[4]
quartiles

In [ ]:
# Products above quartile 75
print('Prime products [above quartile 75] -> {} products ({}%) '.format(df_75.Product.to_list().__len__(), round((df_75.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Premium products [between quartiles 50 & 75] -> {} products ({}%) '.format(df_50_75.Product.to_list().__len__(), round((df_50_75.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Mid products [between quartiles 25 & 50] -> {} products ({}%) '.format(df_25_50.Product.to_list().__len__(), round((df_25_50.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Low products [below quartile 25 ] -> {} products ({}%) '.format(df_25.Product.to_list().__len__(), round((df_25.Product.to_list().__len__() / df.__len__()) * 100, 2)))

In [ ]:
print('Quartile (75>) - Products: {}'.format(df_75.__len__()))
display(df_75.head())
print('Quartile (50-75] - Products: {}'.format(df_50_75.__len__()))
display(df_50_75.head())
print('Quartile (25-50] - Products: {}'.format(df_25_50.__len__()))
display(df_25_50.head())
print('Quartile (<25] - Products: {}'.format(df_25.__len__()))
display(df_25.head())

In [ ]:
print(df[df.Price <= 360].shape[0])
px.box(df,'Price', title=f'Price distribution')

In [ ]:
display(px.histogram(df, x='Price', title=f'Price distribution'))
display(px.histogram(df_75, x='Price', title=f'Price distribution - Prime Products', nbins=10, text_auto=True))
display(df_75.describe().round(2))
display(px.histogram(df_50_75, x='Price', title=f'Price distribution - Premium Products', nbins=15, text_auto=True))
display(df_50_75.describe().round(2))
display(px.histogram(df_25_50, x='Price', title=f'Price distribution - Mid Products', nbins=15, text_auto=True))
display(df_25_50.describe().round(2))

In [ ]:
display(df_75.Product.unique().tolist()[:10])
print()
display(df_50_75.Product.unique().tolist()[:10])
print()
display(df_25_50.Product.unique().tolist()[:10])

In [ ]:
#Prime Products
display(df_75.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10))
for i in df_75.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10).Product.tolist():
    print(i)

#Premium Products
display(df_50_75.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10))
for i in df_50_75.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10).Product.tolist():
    print(i)

#Mid Products
display(df_25_50.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10))
for i in df_25_50.Product.value_counts().to_frame().reset_index().rename(columns={'index':'Product','Product':'Results'}).head(10).Product.tolist():
    print(i)

## Brands distribution

In [ ]:
# Look for specif word(s)

brand_producs = []
for i in df['Product']:
    if (re.search(r'pantene.', i, flags=re.IGNORECASE) or re.search(r'nutre.', i, flags=re.IGNORECASE)) and not ((re.search(r'bamb.', i, flags=re.IGNORECASE)) or (re.search(r'blend.', i, flags=re.IGNORECASE)) or (re.search(r'L\'Oréal', i, flags=re.IGNORECASE))): # or (re.search(r'[^bamboo].', i, flags=re.IGNORECASE))
        brand_producs.append(i)


brand_df = df[df.Product.isin(brand_producs)].reset_index(drop=True)

brand_df = brand_df.drop([4,5,7,8,9,10,11,13]).reset_index(drop=True)

brand_df.to_excel('stella_products.xlsx', index=False)
brand_df